# TRABAJO APRENDIZAJE AUTOMÁTICO

Guzmán Muñoz Revuelta

Fernando Caruana del Pino

Esta parte del trabajo se centra en la preparación avanzada de datos para modelos predictivos. Se abordan cuatro aspectos clave: la división del conjunto de datos en train, validation y test, la combinación eficiente de variables, la estandarización después de modificaciones, la codificación precisa de variables y la imputación de valores faltantes. El objetivo es optimizar la eficiencia y precisión de los modelos mediante la gestión adecuada de los datos.

## IMPORTAMOS LIBRERÍAS

In [23]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
# conda install category_encoders
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics

#from sklearn.metrics import plot_confusion_matrix, roc_curve, roc_auc_score
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

from boruta import BorutaPy # !conda install -c conda-forge boruta_py

## Lectura del dataset: train y test

In [2]:
bf_train = pd.read_csv("../data/train_1.csv")
bf_test = pd.read_csv("../data/test_1.csv")

In [3]:
bf_train = bf_train.drop('Unnamed: 0', axis=1)
bf_test = bf_test.drop('Unnamed: 0', axis=1)

In [5]:
bf_test

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,fraud_bool
0,0.6,0.532791,-1,121,50,0.004671,-1.608770,AD,396,4524.562084,...,0,1000,0,0,6.987319,linux,0,1,6,0
1,0.4,0.655692,-1,58,20,0.008313,51.178750,AA,1074,7728.585220,...,0,200,0,0,19.747368,windows,1,1,4,0
2,0.1,0.869829,-1,354,60,0.011981,-0.521211,AB,929,7078.517070,...,0,200,0,0,2.138674,linux,1,1,1,0
3,0.1,0.185111,31,207,20,0.012904,-0.606201,AB,2299,4057.943868,...,0,200,0,0,5.677184,linux,1,1,1,0
4,0.7,0.124151,-1,57,30,0.013645,51.540936,AA,1496,6268.181291,...,0,500,0,0,3.317681,other,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.7,0.856697,-1,63,30,0.013948,35.547833,AA,2388,6060.140270,...,1,200,0,0,7.004461,other,0,1,2,0
199996,0.6,0.515747,-1,256,60,0.023523,-0.966747,AB,1175,6144.561330,...,1,1000,0,0,8.693469,linux,0,1,2,0
199997,0.6,0.315535,30,16,30,8.818207,20.911347,AA,1736,7530.576613,...,1,500,0,1,3.935820,other,0,1,4,0
199998,0.1,0.470439,99,15,50,0.013798,-1.249123,AC,228,6802.065753,...,0,200,1,0,12.205053,linux,0,1,3,0


## Codificación de la variable objetivo y del resto de variables categoricas

In [6]:
bf_train['fraud_bool'].unique()

array([0, 1], dtype=int64)

In [8]:
bf_train['fraud_bool'].value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

0: No fraude

1: Fraude

In [9]:
y_train = bf_train['fraud_bool']
y_test = bf_test['fraud_bool']
X_train = bf_train.drop('fraud_bool',axis=1)
X_test = bf_test.drop('fraud_bool',axis=1)

In [10]:
X_test

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month
0,0.6,0.532791,-1,121,50,0.004671,-1.608770,AD,396,4524.562084,...,11,0,1000,0,0,6.987319,linux,0,1,6
1,0.4,0.655692,-1,58,20,0.008313,51.178750,AA,1074,7728.585220,...,21,0,200,0,0,19.747368,windows,1,1,4
2,0.1,0.869829,-1,354,60,0.011981,-0.521211,AB,929,7078.517070,...,1,0,200,0,0,2.138674,linux,1,1,1
3,0.1,0.185111,31,207,20,0.012904,-0.606201,AB,2299,4057.943868,...,28,0,200,0,0,5.677184,linux,1,1,1
4,0.7,0.124151,-1,57,30,0.013645,51.540936,AA,1496,6268.181291,...,10,0,500,0,0,3.317681,other,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,0.7,0.856697,-1,63,30,0.013948,35.547833,AA,2388,6060.140270,...,1,1,200,0,0,7.004461,other,0,1,2
199996,0.6,0.515747,-1,256,60,0.023523,-0.966747,AB,1175,6144.561330,...,20,1,1000,0,0,8.693469,linux,0,1,2
199997,0.6,0.315535,30,16,30,8.818207,20.911347,AA,1736,7530.576613,...,3,1,500,0,1,3.935820,other,0,1,4
199998,0.1,0.470439,99,15,50,0.013798,-1.249123,AC,228,6802.065753,...,-1,0,200,1,0,12.205053,linux,0,1,3


In [15]:
y_test

0         0
1         0
2         0
3         0
4         0
         ..
199995    0
199996    0
199997    0
199998    0
199999    0
Name: fraud_bool, Length: 200000, dtype: int64

In [11]:
X_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,bank_months_count,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month
0,0.1,0.220880,-1,11,20,0.008167,-0.873330,AB,3503,11086.005659,...,28,0,200,0,0,5.820871,other,1,1,0
1,0.6,0.223695,27,11,30,0.015370,26.471455,AA,1375,10513.330072,...,21,1,500,0,0,51.423691,linux,0,1,3
2,0.2,0.630280,-1,36,20,0.012853,17.774441,AB,1274,1964.345725,...,31,0,200,0,0,6.614004,linux,0,1,1
3,0.1,0.833989,59,3,30,65.743898,-0.837404,AB,1501,5894.569722,...,21,1,200,0,0,4.122503,other,1,1,2
4,0.4,0.206311,-1,52,40,0.002940,-1.127592,AD,1746,4376.890870,...,1,0,200,0,0,3.014726,other,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,0.6,0.235605,-1,102,40,0.043200,-1.140572,AB,1238,5516.993271,...,4,0,200,0,0,4.199684,other,0,1,5
799996,0.7,0.217125,-1,32,40,0.035689,-1.077081,AC,394,7201.006731,...,-1,0,200,0,0,1.731171,other,1,1,5
799997,0.9,0.827970,-1,128,40,0.018614,-0.804419,AB,904,3368.046158,...,1,0,1500,0,0,4.120950,windows,0,1,5
799998,0.7,0.402069,-1,41,40,0.010033,-0.569603,AC,2278,5923.401231,...,-1,0,200,0,0,7.906814,other,1,1,2


Hacemos lo mismo para la variable y_train:

In [12]:
y_train.value_counts()

fraud_bool
0    791177
1      8823
Name: count, dtype: int64

 # Codificación de variables categóricas

In [13]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

In [14]:
model

OneHotEncoder(cols=['payment_type', 'employment_status', 'housing_status',
                    'device_os'])

In [15]:
X_train_t = model.transform(X_train)
X_test_t = model.transform(X_test)

In [16]:
X_train_t

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type_1,payment_type_2,payment_type_3,...,source,session_length_in_minutes,device_os_1,device_os_2,device_os_3,device_os_4,device_os_5,keep_alive_session,device_distinct_emails_8w,month
0,0.1,0.220880,-1,11,20,0.008167,-0.873330,1,0,0,...,0,5.820871,1,0,0,0,0,1,1,0
1,0.6,0.223695,27,11,30,0.015370,26.471455,0,1,0,...,0,51.423691,0,1,0,0,0,0,1,3
2,0.2,0.630280,-1,36,20,0.012853,17.774441,1,0,0,...,0,6.614004,0,1,0,0,0,0,1,1
3,0.1,0.833989,59,3,30,65.743898,-0.837404,1,0,0,...,0,4.122503,1,0,0,0,0,1,1,2
4,0.4,0.206311,-1,52,40,0.002940,-1.127592,0,0,1,...,0,3.014726,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,0.6,0.235605,-1,102,40,0.043200,-1.140572,1,0,0,...,0,4.199684,1,0,0,0,0,0,1,5
799996,0.7,0.217125,-1,32,40,0.035689,-1.077081,0,0,0,...,0,1.731171,1,0,0,0,0,1,1,5
799997,0.9,0.827970,-1,128,40,0.018614,-0.804419,1,0,0,...,0,4.120950,0,0,1,0,0,0,1,5
799998,0.7,0.402069,-1,41,40,0.010033,-0.569603,0,0,0,...,0,7.906814,1,0,0,0,0,1,1,2


In [17]:
len(list(X_train_t.columns))

50

In [18]:
X_train_t.columns

Index(['income', 'name_email_similarity', 'prev_address_months_count',
       'current_address_months_count', 'customer_age', 'days_since_request',
       'intended_balcon_amount', 'payment_type_1', 'payment_type_2',
       'payment_type_3', 'payment_type_4', 'payment_type_5', 'zip_count_4w',
       'velocity_6h', 'velocity_24h', 'velocity_4w', 'bank_branch_count_8w',
       'date_of_birth_distinct_emails_4w', 'employment_status_1',
       'employment_status_2', 'employment_status_3', 'employment_status_4',
       'employment_status_5', 'employment_status_6', 'employment_status_7',
       'credit_risk_score', 'email_is_free', 'housing_status_1',
       'housing_status_2', 'housing_status_3', 'housing_status_4',
       'housing_status_5', 'housing_status_6', 'housing_status_7',
       'phone_home_valid', 'phone_mobile_valid', 'bank_months_count',
       'has_other_cards', 'proposed_credit_limit', 'foreign_request', 'source',
       'session_length_in_minutes', 'device_os_1', 'device_os_

## Escalado de variables

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_t)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_t), columns=X_train_t.columns, index=X_train_t.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_t), columns=X_test_t.columns, index=X_test.index)

In [20]:
X_train_scaled.describe()

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type_1,payment_type_2,payment_type_3,...,source,session_length_in_minutes,device_os_1,device_os_2,device_os_3,device_os_4,device_os_5,keep_alive_session,device_distinct_emails_8w,month
count,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,...,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05,8.000000e+05
mean,-3.119993e-16,1.302869e-16,-4.510170e-17,-9.485746e-17,-6.885159e-17,5.014655e-17,-1.071676e-16,-1.092459e-17,9.471535e-17,-5.044409e-17,...,4.678924e-17,-5.506706e-19,-6.741274e-17,-1.161737e-17,1.254108e-17,-2.668088e-17,6.594725e-18,1.024336e-16,-6.930456e-17,-5.252687e-17
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,...,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.593774e+00,-1.707812e+00,-4.021494e-01,-9.910460e-01,-1.969858e+00,-1.907973e-01,-1.196036e+00,-7.667177e-01,-5.910854e-01,-3.668531e-01,...,-8.400566e-02,-1.063226e+00,-7.224079e-01,-7.056562e-01,-5.979566e-01,-2.389281e-01,-8.563520e-02,-1.167442e+00,-1.117104e+01,-1.487683e+00
25%,-9.047128e-01,-9.293343e-01,-4.021494e-01,-7.647670e-01,-1.138208e+00,-1.894607e-01,-4.871886e-01,-7.667177e-01,-5.910854e-01,-3.668531e-01,...,-8.400566e-02,-5.529117e-01,-7.224079e-01,-7.056562e-01,-5.979566e-01,-2.389281e-01,-8.563520e-02,-1.167442e+00,-1.009934e-01,-1.035156e+00
50%,1.288785e-01,-4.784795e-03,-4.021494e-01,-3.914066e-01,-3.065576e-01,-1.879799e-01,-4.698547e-01,-7.667177e-01,-5.910854e-01,-3.668531e-01,...,-8.400566e-02,-3.025825e-01,-7.224079e-01,-7.056562e-01,-5.979566e-01,-2.389281e-01,-8.563520e-02,8.565735e-01,-1.009934e-01,-1.301015e-01
75%,8.179393e-01,9.057985e-01,-1.071174e-01,4.910816e-01,5.250924e-01,-1.859042e-01,-1.671058e-01,1.304261e+00,1.691803e+00,-3.668531e-01,...,-8.400566e-02,1.642660e-01,1.384260e+00,1.417121e+00,1.672362e+00,-2.389281e-01,-8.563520e-02,8.565735e-01,-1.009934e-01,7.749526e-01
max,1.162470e+00,1.749356e+00,8.267252e+00,3.862639e+00,4.683343e+00,1.438477e+01,5.151240e+00,1.304261e+00,1.691803e+00,2.725887e+00,...,1.190396e+01,9.752364e+00,1.384260e+00,1.417121e+00,1.672362e+00,4.185359e+00,1.167744e+01,8.565735e-01,5.434030e+00,1.680007e+00


In [21]:
#Guardar los csv
X_train_scaled.to_csv("../data/X_train_scaled.csv", index=False)
X_test_scaled.to_csv("../data/X_test_scaled.csv", index=False)

In [22]:
y_train.to_csv("../data/y_train.csv")
y_test.to_csv("../data/y_test.csv")

## IMPUTACIÓN DE MISSINGS

La imputación de missings ha sido explicada en la parte del EDA, en la interpretación de valores nulos. Como resumen general, la cantidad de nulos encontrados en las siguientes variables es:

- La cantidad de valores nulos en **'prev_address_months_count'** es: 712920

- La cantidad de valores nulos en **'current_address_months_count'** es: 4254

- La cantidad de valores nulos en **' intended_balcon_amount'** es: 742523

- La cantidad de valores nulos en **'bank_months_count'** es: 253635

- La cantidad de valores nulos en **'session_length_in_minutes'** es: 2015

- La cantidad de valores nulos en **'device_distinct_emails '** es: 359